Min Cost Max Flow

In [26]:
# haven't done this shit in forever fuck

# while True:
# we find augmenting path in residual graph
# break if none
# add bottleneck to max flow and update path
# return max flow

import dataclasses
from typing import List, Set, Dict, Hashable
import collections
import math

Node = Hashable

@dataclasses.dataclass(kw_only=True)
class Edge:
    fro: Node
    to: Node
    capacity: float
    flow: float = 0
    cost: float = 0

    def remaining_capacity(self) -> float:
        return self.capacity - self.flow

@dataclasses.dataclass
class NetworkEdge:
    original: Edge
    residual: Edge 

    def augment(self, bottle_neck):
        self.original.flow += bottle_neck
        self.residual.flow -= bottle_neck

@dataclasses.dataclass
class Network:
    source: Node
    sink: Node
    nodes: Set[Node]
    graph: Dict[Node, List[NetworkEdge]]

def create_network(source: Node, sink: Hashable, edges: List[Edge]):
    nodes = {edge.fro for edge in edges} | {edge.to for edge in edges} 
    graph = collections.defaultdict(list)
    for edge in edges:
        residual = Edge(fro=edge.to, to=edge.fro, capacity=0, cost=-edge.cost)
        graph[edge.fro].append(NetworkEdge(edge, residual))
        graph[edge.to].append(NetworkEdge(residual, edge))
    return Network(source, sink, nodes, graph)


def get_augmenting_path_with_dfs(network: Network, visited: Set, path_so_far: List[NetworkEdge] = None):
    path_so_far = path_so_far if path_so_far is not None else []
    node = path_so_far[-1].original.to if path_so_far else network.source
    if node == network.sink:
        return path_so_far

    visited.add(node)
    for network_edge in network.graph[node]:
        if network_edge.original.remaining_capacity() > 0 and network_edge.original.to not in visited:
            path_so_far.append(network_edge)
            path = get_augmenting_path_with_dfs(network, visited, path_so_far)
            if path:
                return path
            path_so_far.pop()
    return []


@dataclasses.dataclass
class NetworkFlowSolution:
    max_flow: float
    min_cut: Set[Node]
    

def augment_until_solved(network: Network, get_augmenting_path=get_augmenting_path_with_dfs) -> NetworkFlowSolution:
    max_flow = 0
    while True:
        visited = set()
        augmenting_path = get_augmenting_path(network, visited)
        if not augmenting_path:
            break
        bottle_neck = min(network_edge.original.remaining_capacity() for network_edge in augmenting_path) 
        for edge in augmenting_path:
            edge.augment(bottle_neck)
        max_flow += bottle_neck
    return NetworkFlowSolution(max_flow, visited)

import pprint

source, sink = 10, 11

edges = [
 # edges from source
 Edge(fro=source, to=0, capacity=10),
 Edge(fro=source, to=1, capacity=5),
 Edge(fro=source, to=2, capacity=10),

 # middle edges
 Edge(fro=0, to=3, capacity=10),
 Edge(fro=1, to=2, capacity=10),
 Edge(fro=2, to=5, capacity=15),
 Edge(fro=3, to=1, capacity=2),
 Edge(fro=3, to=6, capacity=15),
 Edge(fro=4, to=1, capacity=15),
 Edge(fro=4, to=3, capacity=3),
 Edge(fro=5, to=4, capacity=4),
 Edge(fro=5, to=8, capacity=10),
 Edge(fro=6, to=7, capacity=10),
 Edge(fro=7, to=4, capacity=10),
 Edge(fro=7, to=5, capacity=7),

 # edges to sink
 Edge(fro=6, to=sink, capacity=15),
 Edge(fro=8, to=sink, capacity=10),

]
my_network = create_network(source, sink, edges)
pprint.pprint(my_network.nodes)
pprint.pprint(augment_until_solved(my_network))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11}
NetworkFlowSolution(max_flow=23, min_cut={1, 2, 4, 5, 10})
